# MLflow Registry Tutorial

[Tutorial from Official Website](https://mlflow.org/docs/latest/model-registry.html) 

## Working with Models in a Registry
For this to work, you'll need to point MLflow to a database and path. When running locally, you must do this otherwise you'll get into error conditions.

Start a server with:

mlflow server --backend-store-uri sqlite:///mlflow.db --default-artifact-root /tmp/ --host 127.0.0.1:5000
The value of --host must match the tracking uri in the next cell.

In [2]:

import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")

## Method 1 Direct way to register model

In [3]:
from sklearn.datasets import make_regression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
import mlflow.sklearn
from mlflow.models import infer_signature

with mlflow.start_run() as run:
    X, y = make_regression(n_features=4, n_informative=2, random_state=0, shuffle=False)
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    params = {"max_depth": 2, "random_state": 42}
    model = RandomForestRegressor(**params)
    model.fit(X_train, y_train)

    # Infer the model signature
    y_pred = model.predict(X_test)
    signature = infer_signature(X_test, y_pred)

    # Log parameters and metrics using the MLflow APIs
    mlflow.log_params(params)
    mlflow.log_metrics({"mse": mean_squared_error(y_test, y_pred)})

    # Log the sklearn model and register as version 1
    mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="sklearn-model",
        signature=signature,
        registered_model_name="sk-learn-random-forest-reg-model",
    )

/opt/conda/envs/mlflow/lib/python3.10/site-packages/_distutils_hack/__init__.py:18: UserWarning: Distutils was imported before Setuptools, but importing Setuptools also replaces the `distutils` module in `sys.modules`. This may lead to undesirable behaviors or errors. To avoid these issues, avoid using distutils directly, ensure that setuptools is installed in the traditional way (e.g. not an editable install), and/or make sure that setuptools is always imported before distutils.
  warnings.warn(
/opt/conda/envs/mlflow/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
Successfully registered model 'sk-learn-random-forest-reg-model'.
2023/11/12 05:16:32 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: sk-learn-random-forest-reg-model, version 1
Created version '1' of model 'sk-learn-random-forest-reg-

In [4]:
import time
from mlflow.tracking.client import MlflowClient
from mlflow.entities.model_registry.model_version_status import ModelVersionStatus
 
def wait_until_ready(model_name, model_version):
  client = MlflowClient()
  for _ in range(10):
    model_version_details = client.get_model_version(
      name=model_name,
      version=model_version,
    )
    status = ModelVersionStatus.from_string(model_version_details.status)
    print("Model status: %s" % ModelVersionStatus.to_string(status))
    if status == ModelVersionStatus.READY:
      break
    time.sleep(1)
  
wait_until_ready("sk-learn-random-forest-reg-model", 1)

Model status: READY


## Fetching an MLflow Model from the Model Registry


In [5]:
import mlflow.pyfunc

model_name = "sk-learn-random-forest-reg-model"
model_version = 1

model = mlflow.pyfunc.load_model(model_uri=f"models:/{model_name}/{model_version}")

# model.predict(data)

In [19]:
# try to make a data for prediction similart to X_test
import pandas as pd
import numpy as np
noise = np.random.normal(0, 0.1, X_test.shape)
data = X_test + noise

print(data)



[[ 0.7738581  -1.20190728 -2.71964879  0.80028994]
 [ 0.87788274 -0.83588386 -1.03949091 -0.33129127]
 [-0.20492005  1.04839896 -0.71305165  1.64979906]
 [-0.23981533 -1.27459505 -0.66418556 -2.31635217]
 [-0.73415699 -0.81064163 -0.51872352  0.02460382]
 [ 1.20778904 -1.07123529 -1.04032984 -0.54443559]
 [-1.0258869   1.03726463 -0.29417252  1.22435456]
 [-0.64713849 -0.79610114  0.38344293 -0.94454131]
 [-1.19051671 -1.39261389 -1.64568994  1.91243474]
 [ 1.77273421  0.29911469  1.00751633  2.28238201]
 [ 1.18417582 -1.22962802  0.46448645 -0.6374576 ]
 [ 0.4627346  -1.20845683  0.29344062  1.27780249]
 [ 2.25848083 -1.07895436 -0.32857996  1.23431573]
 [-0.70029994 -0.11980828 -0.64808019  0.63401651]
 [ 0.58662111 -0.07070207  0.84085923  0.2623277 ]
 [ 1.55308344 -0.2266756   0.27968644 -0.76433406]
 [ 0.66499934 -0.41181947 -1.37335907 -1.21730891]
 [ 0.55151743 -0.09227321 -0.10787233  0.83602366]
 [-1.65679856  0.06914491 -1.04257839  0.42164761]
 [-0.70285729 -0.18010124 -0.44

In [22]:
model.predict(data)

array([-25.73795219, -21.26614791,  43.72788113, -37.62039689,
       -35.30388767, -23.72605273,  41.1213532 , -35.16879842,
       -40.16931792,  15.12985201, -24.2475118 , -28.49126919,
       -23.62019943, -10.99985266,  -2.28963784,  -3.92252345,
       -15.54128375,  -2.28963784, -11.95702358, -11.5213699 ])